## Early Fusion (Concate)
Disini bakal ngeconcate data dari :
1. TF-IDF (teks) 
2. MobileNetV1 (gambar)
3. Feature Metadata (numerik)

#### Import Library

In [1]:
import os
import numpy as np
from scipy import sparse

#### Load Artifact 

In [ ]:
def load_all_features(base_path, ratio_str):
    """Memuat semua matriks fitur (teks, gambar, metadata) untuk train dan test."""
    
    # Tentukan path dinamis berdasarkan rasio
    tfidf_path = os.path.join(base_path, f'_tfidf_artifacts/unigram_{ratio_str}')
    image_path = os.path.join(base_path, f'_image_artifacts/mobilenetv1_{ratio_str}')
    meta_path = os.path.join(base_path, f'_feature_artifacts/metadata_{ratio_str}')
    
    try:
        # Muat fitur train
        X_train_text = sparse.load_npz(os.path.join(tfidf_path, 'X_train_unigram.npz'))
        X_train_image = np.load(os.path.join(image_path, 'X_train_image.npy'))
        X_train_meta = np.load(os.path.join(meta_path, 'X_train_metadata.npy'))
        
        # Muat fitur test
        X_test_text = sparse.load_npz(os.path.join(tfidf_path, 'X_test_unigram.npz'))
        X_test_image = np.load(os.path.join(image_path, 'X_test_image.npy'))
        X_test_meta = np.load(os.path.join(meta_path, 'X_test_metadata.npy'))
        
        print("Semua matriks fitur berhasil dimuat.")
        print("tfidf : ", X_train_text.shape, X_test_text.shape)
        print("image : ", X_train_image.shape, X_test_image.shape)
        print("meta  : ", X_train_meta.shape, X_test_meta.shape)
        return (X_train_text, X_train_image, X_train_meta, 
                X_test_text, X_test_image, X_test_meta)
        
        
    except FileNotFoundError as e:
        print(f"Error: Gagal memuat file. Pastikan semua path benar. {e}")
        return (None,) * 6

# --- Konfigurasi dan Eksekusi ---
BASE_DATA_DIR = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA'
SPLIT_RATIO = '8020'

(X_train_text, X_train_image, X_train_meta, 
 X_test_text, X_test_image, X_test_meta) = load_all_features(BASE_DATA_DIR, SPLIT_RATIO)

Semua matriks fitur berhasil dimuat.


#### Fusion (Concate)

In [3]:
def fuse_features(text_features, image_features, meta_features):
    """Menggabungkan tiga matriks fitur secara horizontal."""
    
    print("Memulai proses fusion...")
    
    # Gunakan hstack untuk menggabungkan sparse (TF-IDF) dan dense (gambar, meta)
    fused_matrix = sparse.hstack([text_features, image_features, meta_features])
    
    print(f"Fusion berhasil. Bentuk matriks akhir: {fused_matrix.shape}")
    return fused_matrix

# --- Eksekusi ---
if X_train_text is not None:
    # Fusion untuk data latih
    X_train_fused = fuse_features(X_train_text, X_train_image, X_train_meta)
    
    # Fusion untuk data uji
    X_test_fused = fuse_features(X_test_text, X_test_image, X_test_meta)

Memulai proses fusion...
Fusion berhasil. Bentuk matriks akhir: (13044, 18221)
Memulai proses fusion...
Fusion berhasil. Bentuk matriks akhir: (3261, 18221)


#### Save Files

In [ ]:
def save_fused_data(train_fused, test_fused, output_dir, ratio_str):
    """Menyimpan matriks fitur yang sudah digabungkan."""
    
    os.makedirs(output_dir, exist_ok=True)
    
    train_path = os.path.join(output_dir, f'X_train_fused_{ratio_str}.npz')
    test_path = os.path.join(output_dir, f'X_test_fused_{ratio_str}.npz')
    
    # Simpan sebagai sparse matrix (.npz) karena efisien
    sparse.save_npz(train_path, train_fused)
    sparse.save_npz(test_path, test_fused)
    
    print(f"Matriks fused train berhasil disimpan di: {train_path}")
    print(f"Matriks fused test berhasil disimpan di: {test_path}")

# --- Konfigurasi dan Eksekusi ---
FUSED_OUTPUT_DIR = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\_$fused'

if 'X_train_fused' in locals():
    save_fused_data(X_train_fused, X_test_fused, FUSED_OUTPUT_DIR, SPLIT_RATIO)

Matriks fused train berhasil disimpan di: E:\$7th\TA\Eksploring_TF-IDF\DATA\_$fused\X_train_fused_8020.npz
Matriks fused test berhasil disimpan di: E:\$7th\TA\Eksploring_TF-IDF\DATA\_$fused\X_test_fused_8020.npz
